In [ ]:
import inro.modeller as _m
import csv
import os
import multiprocessing
import numpy as np
import pandas as pd
import sqlite3
import datetime
import traceback as _traceback

In [ ]:
dt = _m.Modeller().desktop
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()

# Get List of Scenarios from Desktop
### Make sure spelling is correct in params dictionary below or run will FAIL!!!

In [ ]:
# make sure modeller is closed or it will print to the python console in there
counter = 0
for eb in ebs:
    print '{} {}'.format(counter, eb.title())
    counter += 1

# Set Parameters

In [ ]:
params = {
    'run2017': {
        'master_scen': 1700,
        'master_scen_name': "BAU_run2017",
        'scenario_name': 'BusinessAsUsual',
        'alternative_name':  'run2016',
        'horizon_year': 2017,
        'global_iterations': 4,
        'max_distribution_iterations': 60,
        'max_assignment_iterations': 200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': True
    },
    'run2035': {
        'master_scen': 3500,
        'master_scen_name': "BAU_run2035",
        'scenario_name': 'BusinessAsUsual',
        'alternative_name':  'run2035',
        'horizon_year': 2035,
        'global_iterations': 4,
        'max_distribution_iterations': 60,
        'max_assignment_iterations': 200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': True},
    'run2050': {
        'master_scen': 5000,
        'master_scen_name': "BAU_run2050",
        'scenario_name': 'BusinessAsUsual',
        'alternative_name':  'run2050',
        'horizon_year': 2050,
        'global_iterations': 4,
        'max_distribution_iterations': 60,
        'max_assignment_iterations': 200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': True
    }
}

# Loop through and run all the ebs open in desktop (except minimal base) 

In [ ]:
run_model = _m.Modeller().tool("translink.RTM3.runmodel")
util = _m.Modeller().tool("translink.util")

In [ ]:
for eb in ebs:
    title = eb.title()
    
    # do not run minimal base databank
    if title == 'Minimal Base Databank':
        continue

    # do not run scenario not specified
    if title not in params:
        continue
    
    eb.open()
    
    current_time = datetime.datetime.now()
    print "\nstarting {} run at {}".format(title, current_time)
    
    try:    
        year = params[title]['horizon_year']
        ip = util.get_input_path(eb)
        geoFile = os.path.join(ip, 'taz1700_geographics_{}.csv'.format(year))
        demoFile = os.path.join(ip, 'taz1700_demographics_{}.csv'.format(year))
        scen = eb.scenario_by_number(params[title]['master_scen']).core_scenario
        
        
        run_model(scenario_name = params[title]['scenario_name'],
                  alternative_name = params[title]['alternative_name'], 
                  horizon_year=year, 
                  global_iterations=params[title]['global_iterations'], 
                  master_scen = scen, 
                  demographics_file = demoFile,
                  geographics_file = geoFile, 
                  max_distribution_iterations = params[title]['max_distribution_iterations'],
                  max_assignment_iterations = params[title]['max_assignment_iterations'], 
                  run_congested_transit = params[title]['run_congested_transit'],
                  run_capacited_transit = params[title]['run_capacited_transit'], 
                  num_processors = params[title]['num_processors'], 
                  run_parking_model = params[title]['run_parking_model'],
                  run_toll_skim = params[title]['run_toll_skim'])
        current_time = datetime.datetime.now()
        print "{} ran successfully at {}".format(title, current_time)
        
        
    except Exception as e:
        current_time = datetime.datetime.now()
        print("Scenario {} run failed at {}.".format(title, current_time), e)
